In [11]:
import pydae.build_cffi as db
from pydae.bmapu import bmapu_builder
from pydae.build_v2 import builder


## PV plant

![Alt text](pv_nm.svg)

A PV with converted connected to an infinite bus through a line.


![Alt text](spvib.svg)
![Alt text](pv_module.svg)



### Option 1: using a python dic to define the system

In [12]:
N = 3
M = 2
S_pv_mva = 1.0

data = {
    "system":{"name":f"pv_{M}_{N}","S_base":100e6,"K_p_agc":0.0,"K_i_agc":0.0,"K_xif":0.01},
    "buses":[
        {"name":"POI_MV","P_W":0.0,"Q_var":0.0,"U_kV":20.0},
        {"name":   "POI","P_W":0.0,"Q_var":0.0,"U_kV":132.0},
        {"name":  "GRID","P_W":0.0,"Q_var":0.0,"U_kV":132.0}
    ],
    "lines":[
        {"bus_j":"POI_MV","bus_k": "POI","X_pu":0.05,"R_pu":0.0,"Bs_pu":0.0,"S_mva":120},
        {"bus_j":   "POI","bus_k":"GRID","X_pu":0.02,"R_pu":0.0,"Bs_pu":0.0,"S_mva":120, 'sym':True, 'monitor':True}
        ],
    "pvs":[],
    "genapes":[{
          "bus":"GRID","S_n":1000e6,"F_n":50.0,"X_v":0.001,"R_v":0.0,
          "K_delta":0.001,"K_alpha":1e-6}]
    }

for i_m in range(1,M+1):
    name_j = "POI_MV"
    for i_n in range(1,N+1):
        name = f"{i_m}".zfill(2) + f"{i_n}".zfill(2)
        name_k = 'MV' + name

        data['buses'].append({"name":f"LV{name}","P_W":0.0,"Q_var":0.0,"U_kV":0.4})
        data['buses'].append({"name":f"MV{name}","P_W":0.0,"Q_var":0.0,"U_kV":20.0})

        data['lines'].append({"bus_j":f"LV{name}","bus_k":f"MV{name}","X_pu":0.05,"R_pu":0.0,"Bs_pu":0.0,"S_mva":1.2*S_pv_mva,"monitor":False})
        data['lines'].append({"bus_j":f"{name_k}","bus_k":f"{name_j}","X_pu":0.02,"R_pu":0.0,"Bs_pu":0.0,"S_mva":1.2*S_pv_mva*(N-i_n+1),"monitor":False})
        name_j = name_k
        data['pvs'].append({"bus":f"LV{name}","type":"pv_dq","S_n":S_pv_mva*1e6,"U_n":400.0,"F_n":50.0,"X_s":0.1,"R_s":0.01,"monitor":False,
                            "I_sc":8,"V_oc":42.1,"I_mp":3.56,"V_mp":33.7,"K_vt":-0.160,"K_it":0.065,"N_pv_s":25,"N_pv_p":250})
    


In [13]:
data

{'system': {'name': 'pv_2_3',
  'S_base': 100000000.0,
  'K_p_agc': 0.0,
  'K_i_agc': 0.0,
  'K_xif': 0.01},
 'buses': [{'name': 'POI_MV', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 20.0},
  {'name': 'POI', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 132.0},
  {'name': 'GRID', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 132.0},
  {'name': 'LV0101', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 0.4},
  {'name': 'MV0101', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 20.0},
  {'name': 'LV0102', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 0.4},
  {'name': 'MV0102', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 20.0},
  {'name': 'LV0103', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 0.4},
  {'name': 'MV0103', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 20.0},
  {'name': 'LV0201', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 0.4},
  {'name': 'MV0201', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 20.0},
  {'name': 'LV0202', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 0.4},
  {'name': 'MV0202', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 20.0},
  {'name': 'LV0203', 'P_W': 0.0, 'Q_var': 0.0, 'U_kV': 0.4},
  {'name': 'MV0203', 'P

### Option 2: using a .json file or a .hjson file

In [14]:
#grid = bmapu_builder.bmapu('spvib.json')
grid = bmapu_builder.bmapu(data)

grid.uz_jacs = False
grid.verbose = True
grid.construct(f'pv_{M}_{N}')

In [15]:
b = builder(grid.sys_dict,verbose=True)
b.dict2system()
b.functions()
b.jacobians()
b.cwrite()
b.template()
b.compile()

2023-05-18 13:50:52,737 check_system
2023-05-18 13:50:52,761 computing jacobians Fx_run,Fy_run
2023-05-18 13:50:52,768 computing jacobians Gx_run,Gy_run
2023-05-18 13:50:53,306 computing jacobians Fu_run,Gu_run
2023-05-18 13:50:53,613 computing jacobians Fx_ini,Fy_ini
2023-05-18 13:50:53,617 computing jacobians Gy_ini
2023-05-18 13:50:54,005 computing jacobians Hx_run,Hy_run,Hu_run
2023-05-18 13:50:54,103 end checking and computing jacobians
2023-05-18 13:50:54,104 f_ini symbolic to c and xyup
2023-05-18 13:50:54,108 end f_ini symbolic to c and xyup
2023-05-18 13:50:54,109 f_run symbolic to c and xyup
2023-05-18 13:50:54,114 end f symbolic to c and xyup
2023-05-18 13:50:54,115 g_ini symbolic to c and xyup
2023-05-18 13:50:54,396 end g_ini symbolic to c and xyup
2023-05-18 13:50:54,397 g_run symbolic to c and xyup
2023-05-18 13:50:54,676 end g_run symbolic to c and xyup
2023-05-18 13:50:54,678 h symbolic to c and xyup
2023-05-18 13:50:54,698 end h symbolic to c and xyup
2023-05-18 13:50